# Python Cryptography Tutorial 

#### Amirali Sanatinia (amirali@ccs.neu.edu)
##### Network Security (CS 6740)

There are a number of crypto libraries in Python, (cryptography, pycrypto, m2crypto). In this tutorial we will cover cryptography. You can download the library from [here](https://cryptography.io/) and follow the instructions. You should be able to install the library using the following command. You need to have pip insalled. 

``pip install cryptography``

To install pip, follow the instructions [here](https://pip.pypa.io/)

Cryptography components are divided into different submodules. Following is a list of these submodules (not exhaustive)

* Primitive Crypto Blocks (*cryptography.hazmat*)
 * Message Digest and Hashing algorithms (*cryptography.hazmat.primitives.hashes*)
 * Symmetric encryption algorithms (*cryptography.hazmat.primitives.ciphers*)
 * Asymmetric encryption algorithms (*cryptography.hazmat.primitives.asymmetric*)
* X.509 Ecosystem (*cryptography.x509*)
* Full high level crypto recipe (*cryptography.fernet*)

## Hashing Algorithms

As we discussed in class, the goal is to have a long message as input and produce an output which is much shorter called the hash or message digest. Furthermore, we want it to have properties such as pre-image, second preimage, and  collision resistance. *MD5* and *SHA* hash families are such examples.

### MD5

MD5 is hashing algorithm with block size of 512 bits, and digest size of 128 bit. It was designed by Ron Rivest in 1991. MD5 is considered insecure these days, and it's highly advised not to be used in new systems.

In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
digest = hashes.Hash(hashes.MD5(), backend=default_backend())
digest.update(b"Network")
digest.update(b"Security")
msg_digest = digest.finalize()

In [2]:
# Notice the output size of the digest
print "MD5", len(msg_digest), len(msg_digest) * 8

MD5 16 128


### SHA Family

Secure Hash Algorithm (SHA) family, is a series of hashing algorithms. Ranging from SHA-0 to SHA-3. SHA-0 should never be used, it's advised to move from SHA-1 to SHA-2. SHA-3 is the most recent version, published in 2015.

 * SHA-1: Digest size (160), Block size (512)
 * SHA-2: Digest size (224, 256, 384, or 512), Block size (512, 1024)
 * SHA-3: Digest size (224, 256, 384, 512), Block size (1600)

In [3]:
for _hash in [hashes.SHA1, hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    digest = hashes.Hash(_hash(), backend=default_backend())
    digest.update(b"Network")
    digest.update(b"Security")
    msg_digest = digest.finalize()
    # Notice the output size of the digest
    print _hash.name, len(msg_digest), len(msg_digest) * 8

sha1 20 160
sha224 28 224
sha256 32 256
sha384 48 384
sha512 64 512


### Hash-based message authentication code (HMAC)

HMACs are used for message authentications combined with a secret key. The provide integrity check and authentication.

![Alt text](SHAhmac.png)

image source: wikipedia

In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
import os
for _hash in [hashes.SHA1, hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    h = hmac.HMAC(os.urandom(16), _hash(), backend=default_backend())
    h.update("Network Security")
    msg_digest = h.finalize()
    print _hash.name, len(msg_digest), len(msg_digest) * 8

sha1 20 160
sha224 28 224
sha256 32 256
sha384 48 384
sha512 64 512


## Symmetric Encryption

In the following we look at the symmetric encryption algorithms. In symmetric crpto, we use the same key for encryption and decryption. Therefore, the two parties needs to establish a secret key between them. It's up to 1000 times faster than asymmetric encryption.

### Advanced Encryption Algorithm (AES)

AES is based on Rijndael encryption algorithm, designed by Joan Daemen and Vincent Rijmen. It was one of the algorithms submitted to U.S. National Institute of Standards and Technology (NIST) to replace DES and 3DES. It was published in 1998 and accepted and standardized in 2001.

 * AES supports key sizes of 128/192/256 bits
 * Block size: 128 bit
 * It's iterative rather than Feistel cipher
 * Treats data in 4 groups of 4 bytes
 * Operates on an entire block in every round
 * Resistant against known attacks
 * Speed and code compactness on many CPUs
 * Rijndael block and key size vary between 128, 192, 256
 * However, in AES block size in 128
 * Number of rounds a function of key size
  * 128 bits     10 rounds
  * 192 bits     12 rounds
  * 256 bits     14 rounds

 * Today most implementations use the CPU support (Intel AES-NI)

### Block cipher mode of operation

To encrypt messages of arbitrary size with block ciphers, we use the following algorithms, called the modes of operation. They define how to encrypt each block of the plaintext to produce the corresponding cipher text block. Some of these are complemetly insecure (ECB) and should not be used.

 * Electronic Codebook (ECB)
 * Cipher Block Chaining (CBC)
 * Cipher Feedback (CFB)
 * Output Feedback (OFB)
 * Counter (CTR)

The following images are encrypted with ECB. Note that you can see the pattern in the data. Therefore, ECB is not secure or recommended to be used.

![TUX](tux.png)  ![ECB TUX](ECB1.png)  ![ECB TUX](ECB2.png)